In [1]:
import numpy as np
import json
from scipy.sparse import load_npz,save_npz,diags,csr_matrix
import scipy.sparse as sp
import pandas as pd
import os
from io import BytesIO
from tqdm import tqdm
from scipy.sparse.linalg import eigsh
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from pathlib import Path
import igraph as ig
import leidenalg as la
from matplotlib.backends.backend_pdf import PdfPages
from pypdf import PdfReader, PdfWriter
from tempfile import NamedTemporaryFile
import networkx as nx
from sklearn.neighbors import NearestNeighbors
import community as community_louvain
import pickle
from collections import defaultdict

In [2]:
DISEASE = "BIPOLAR"
RESULT_FOLDER = f"../output/{DISEASE}/leiden_results"

In [ ]:
with open(f"{RESULT_FOLDER}/result_communities.pkl", "rb") as f:
    communities = pickle.load(f)

In [ ]:
print(communities)